<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Module 3- Problem #2 What Areas Should the Agency Focus On?


## Problem Statement

<b>Should the Department of Housing Preservation and Development of New York City focus on any particular set of boroughs, ZIP codes, or street (where the complaints are severe) for the specific type of complaints you identified in response to Question 1? </b>

In this exercise, you will use 311 Dataset to determine whether to focus on any particular borough, ZIP code, or street (where the complaints are severe) for the specific Complaint Type you decided to focus at the end of the last exercise.

### 0. Library

In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

def display_n(df,n): 
    with pd.option_context('display.max_rows',n*2):
        display(df)

### 1. Load Data From CSV File

In [2]:


df = pd.read_csv(body)


### Fix formatting and uppercase the certain columns

We could relabel the heating later on after answer the questions

In [3]:
##### Let's clean up the dataset
# drop the first dummy column
df.drop(columns=['Unnamed: 0'], inplace=True)
print("Is Unname column in the dataframe?", ({'Unnamed: 0'}.issubset(df.columns)))

#cast the columns to correct type
# datetime
df['Created Date'] = pd.to_datetime(df['Created Date'], format="%m/%d/%Y %I:%M:%S %p", errors='coerce') #force the invalid to NaT
df['Closed Date'] = pd.to_datetime(df['Closed Date'], format="%m/%d/%Y %I:%M:%S %p", errors='coerce') #force the invalid to NaT

#fix the zipcode - force the nan values to be 0
df['Incident Zip'].replace(np.nan, 0, inplace=True) # let replace all of null with 0
df['Incident Zip'] = df['Incident Zip'].astype('int') #NaN are floats, so let cast the entire column as string

# fix the column names
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace(' ', '_')

#Strip leading and trailing whitespaces and upper all of the cell values - Borough was good before
fix_cols = ['Complaint_Type','Location_Type','Incident_Address','Street_Name','Address_Type','City','Status','Borough'  ]
for i, name in enumerate(fix_cols):
    df[name]=df[name].str.upper().str.strip()
    
print("Shape of dataframe 311 = ", df.shape, "\n")
display_n(df,5)

Is Unname column in the dataframe? False
Shape of dataframe 311 =  (6019843, 15) 



,Unique_Key,Created_Date,Closed_Date,Complaint_Type,Location_Type,Incident_Zip,Incident_Address,Street_Name,Address_Type,City,Status,Resolution_Description,Borough,Latitude,Longitude
0,45531130,2020-02-02 06:09:17,NaT,HEAT/HOT WATER,RESIDENTIAL BUILDING,10019,426 WEST 52 STREET,WEST 52 STREET,ADDRESS,NEW YORK,OPEN,The following complaint conditions are still o...,MANHATTAN,40.765132,-73.988993
1,45529784,2020-02-02 14:15:24,NaT,UNSANITARY CONDITION,RESIDENTIAL BUILDING,11204,1751 67 STREET,67 STREET,ADDRESS,BROOKLYN,OPEN,The following complaint conditions are still o...,BROOKLYN,40.618484,-73.992673
2,45527528,2020-02-02 02:27:41,NaT,HEAT/HOT WATER,RESIDENTIAL BUILDING,11372,87-15 37 AVENUE,37 AVENUE,ADDRESS,JACKSON HEIGHTS,OPEN,The following complaint conditions are still o...,QUEENS,40.750269,-73.879432
3,45530329,2020-02-02 12:13:18,NaT,HEAT/HOT WATER,RESIDENTIAL BUILDING,10458,2405 SOUTHERN BOULEVARD,SOUTHERN BOULEVARD,ADDRESS,BRONX,OPEN,The following complaint conditions are still o...,BRONX,40.853773,-73.881558
4,45528814,2020-02-02 13:59:44,NaT,APPLIANCE,RESIDENTIAL BUILDING,11209,223 78 STREET,78 STREET,ADDRESS,BROOKLYN,OPEN,The following complaint conditions are still o...,BROOKLYN,40.629745,-74.030533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6019838,44063685,2019-10-15 10:35:45,2019-10-16 17:55:02,HEAT/HOT WATER,RESIDENTIAL BUILDING,10029,327 EAST 108 STREET,EAST 108 STREET,ADDRESS,NEW YORK,CLOSED,The complaint you filed is a duplicate of a co...,MANHATTAN,40.791359,-73.940184
6019839,44063692,2019-10-15 06:09:28,2019-10-15 16:12:58,HEAT/HOT WATER,RESIDENTIAL BUILDING,10461,3555 BRUCKNER BOULEVARD,BRUCKNER BOULEVARD,ADDRESS,BRONX,CLOSED,The Department of Housing Preservation and Dev...,BRONX,40.847809,-73.827481
6019840,44063724,2019-10-15 10:54:45,2019-10-16 18:14:29,HEAT/HOT WATER,RESIDENTIAL BUILDING,10034,165 SHERMAN AVENUE,SHERMAN AVENUE,ADDRESS,NEW YORK,CLOSED,The complaint you filed is a duplicate of a co...,MANHATTAN,40.864764,-73.922764
6019841,44063726,2019-10-15 14:07:34,2019-10-16 19:10:19,HEAT/HOT WATER,RESIDENTIAL BUILDING,10467,66 WEST GUN HILL ROAD,WEST GUN HILL ROAD,ADDRESS,BRONX,CLOSED,The Department of Housing Preservation and Dev...,BRONX,40.883207,-73.884279


# Let's start answering the questions

### Setup - Select Key Data Elements (KDE) for Quiz 2

In [4]:
df_q2 = df[["Complaint_Type", "Borough","Incident_Zip","Incident_Address","Street_Name", "Latitude","Longitude","Status"]]
display_n(df_q2,5)

,Complaint_Type,Borough,Incident_Zip,Incident_Address,Street_Name,Latitude,Longitude,Status
0,HEAT/HOT WATER,MANHATTAN,10019,426 WEST 52 STREET,WEST 52 STREET,40.765132,-73.988993,OPEN
1,UNSANITARY CONDITION,BROOKLYN,11204,1751 67 STREET,67 STREET,40.618484,-73.992673,OPEN
2,HEAT/HOT WATER,QUEENS,11372,87-15 37 AVENUE,37 AVENUE,40.750269,-73.879432,OPEN
3,HEAT/HOT WATER,BRONX,10458,2405 SOUTHERN BOULEVARD,SOUTHERN BOULEVARD,40.853773,-73.881558,OPEN
4,APPLIANCE,BROOKLYN,11209,223 78 STREET,78 STREET,40.629745,-74.030533,OPEN
...,...,...,...,...,...,...,...,...
6019838,HEAT/HOT WATER,MANHATTAN,10029,327 EAST 108 STREET,EAST 108 STREET,40.791359,-73.940184,CLOSED
6019839,HEAT/HOT WATER,BRONX,10461,3555 BRUCKNER BOULEVARD,BRUCKNER BOULEVARD,40.847809,-73.827481,CLOSED
6019840,HEAT/HOT WATER,MANHATTAN,10034,165 SHERMAN AVENUE,SHERMAN AVENUE,40.864764,-73.922764,CLOSED
6019841,HEAT/HOT WATER,BRONX,10467,66 WEST GUN HILL ROAD,WEST GUN HILL ROAD,40.883207,-73.884279,CLOSED


## Create the frequency by Target Complaint
### Determine which complaint to address

In [5]:
threshold=800000
df_q2_freq = df_q2['Complaint_Type'].value_counts().to_frame("frequency").rename_axis("complaints").reset_index() # Just double checking if there nulls
df_target_complaint = df_q2_freq[df_q2_freq['frequency'] > threshold]
df_target_complaint

,complaints,frequency
0,HEAT/HOT WATER,1261574
1,HEATING,887850


### - To make our life easier, let's just relabel

In [6]:
df_q2['Complaint_Type'].replace('HEATING', 'HEAT/HOT WATER', inplace=True) 
arr = df_q2.Complaint_Type.unique()
sortedArr=np.sort(arr)
print(sortedArr)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


['AGENCY' 'APPLIANCE' 'CONSTRUCTION' 'DOOR/WINDOW' 'ELECTRIC' 'ELEVATOR'
 'FLOORING/STAIRS' 'GENERAL' 'GENERAL CONSTRUCTION' 'HEAT/HOT WATER'
 'HPD LITERATURE REQUEST' 'MOLD' 'NONCONST' 'OUTSIDE BUILDING'
 'PAINT - PLASTER' 'PAINT/PLASTER' 'PLUMBING' 'SAFETY' 'STRUCTURAL'
 'UNSANITARY CONDITION' 'VACANT APARTMENT' 'WATER LEAK']


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #1: </h1>
    <b>  <u>For the complaint types that you selected in the previous module</u> that had at least 800,000 compliants logged, which <font color=blue>borough</font> had the <font color=blue>highest</font> number of complaints submitted?</b>
</div>

#### By Complaint and Zip Code

In [7]:
# Calculate the frequency by Complaint type and Borough
df_CT_Borough = pd.DataFrame({'frequency': df_q2.groupby( [df_q2.Complaint_Type, df_q2.Borough] ).size()}).reset_index() # could use dataframe for a later use

# Calculate the frequency by Complaint type and Borough
df_Heat_Brough = df_CT_Borough[df_CT_Borough['Complaint_Type'] == 'HEAT/HOT WATER']
df_Heat_Brough.reset_index(drop=True, inplace=True)
df_Heat_Brough

,Complaint_Type,Borough,frequency
0,HEAT/HOT WATER,BRONX,606133
1,HEAT/HOT WATER,BROOKLYN,574845
2,HEAT/HOT WATER,MANHATTAN,423039
3,HEAT/HOT WATER,QUEENS,244122
4,HEAT/HOT WATER,STATEN ISLAND,18368
5,HEAT/HOT WATER,UNSPECIFIED,282917


In [8]:
df_Heat_Brough.sort_values(by='frequency', ascending=False).head(1)

,Complaint_Type,Borough,frequency
0,HEAT/HOT WATER,BRONX,606133


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #2: </h1>
<b>  <u>For the complaint types that you selected in the previous module</u> that had at least 800,000 compliants logged, which <font color=blue>borough</font> had the <font color=blue>lowest</font> number of complaints submitted??</b>
</div>

In [9]:
df_Heat_Brough.sort_values(by='frequency', ascending=False).tail(1)

,Complaint_Type,Borough,frequency
4,HEAT/HOT WATER,STATEN ISLAND,18368


In [16]:
df_q2[(df_q2['Borough'] == "UNSPECIFIED") 
      & (df_q2['Complaint_Type'] == 'HEAT/HOT WATER')].head()

,Complaint_Type,Borough,Incident_Zip,Incident_Address,Street_Name,Latitude,Longitude,Status
177399,HEAT/HOT WATER,UNSPECIFIED,10032,524 WEST 173 STREET,WEST 173 STREET,40.843276,-73.935475,OPEN
177400,HEAT/HOT WATER,UNSPECIFIED,11210,1410 NEW YORK AVENUE,NEW YORK AVENUE,40.638253,-73.945326,OPEN
177401,HEAT/HOT WATER,UNSPECIFIED,11210,1404 NEW YORK AVENUE,NEW YORK AVENUE,40.638360,-73.945337,CLOSED
177402,HEAT/HOT WATER,UNSPECIFIED,11235,2750 HOMECREST AVENUE,HOMECREST AVENUE,40.584273,-73.956308,CLOSED
177403,HEAT/HOT WATER,UNSPECIFIED,10029,241 EAST 115 STREET,EAST 115 STREET,40.796851,-73.939540,CLOSED


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #3: </h1>
<b>  <u>For the complaint types that you selected in the previous module</u> that had at least 800,000 compliants logged, which <font color=blue>ZIP code</font> had the  <font color=blue>highest</font> number of complainted submitted?</b>
</div>

In [11]:
# Calculate the frequency by Complaint type and Borough
df_CT_Zip = pd.DataFrame({'frequency': df_q2.groupby( [df_q2.Complaint_Type, df_q2.Incident_Zip] ).size()}).reset_index() # could use dataframe for a later use

# Calculate the frequency by Complaint type and Borough
df_Heat_Zip = df_CT_Zip[df_CT_Zip['Complaint_Type'] == 'HEAT/HOT WATER']
df_Heat_Zip.reset_index(drop=True, inplace=True)
df_Heat_Zip

,Complaint_Type,Incident_Zip,frequency
0,HEAT/HOT WATER,0,19024
1,HEAT/HOT WATER,10001,4140
2,HEAT/HOT WATER,10002,14500
3,HEAT/HOT WATER,10003,10922
4,HEAT/HOT WATER,10004,161
...,...,...,...
189,HEAT/HOT WATER,11692,3189
190,HEAT/HOT WATER,11693,1271
191,HEAT/HOT WATER,11694,4380
192,HEAT/HOT WATER,11697,66


In [12]:
df_Heat_Zip.sort_values(by='frequency', ascending=False).head(1)

,Complaint_Type,Incident_Zip,frequency
127,HEAT/HOT WATER,11226,69041


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #4: </h1>
<b>  <u>For the complaint types that you selected in the previous module</u> that had at least 800,000 compliants logged, the address <font color=blue>89-21 Elmhurst Avenue</font> had the <font color=blue>highest</font> number of complainted submitted? <font color=blue>(True or False)</font></b>
</div>

In [13]:
# Calculate the frequency by Complaint type and Borough
df_CT_Address = pd.DataFrame({'frequency': df_q2.groupby( [df_q2.Complaint_Type, df_q2.Incident_Address] ).size()}).reset_index() # could use dataframe for a later use

# Calculate the frequency by Complaint type and Borough
df_Heat_Add = df_CT_Address[df_CT_Address['Complaint_Type'] == 'HEAT/HOT WATER']
df_Heat_Add.reset_index(drop=True, inplace=True)
df_Heat_Add

,Complaint_Type,Incident_Address,frequency
0,HEAT/HOT WATER,1 1 AVENUE,2
1,HEAT/HOT WATER,1 12 STREET,1
2,HEAT/HOT WATER,1 43 STREET,1
3,HEAT/HOT WATER,1 5 AVENUE,1
4,HEAT/HOT WATER,1 7 AVENUE,4
...,...,...,...
136944,HEAT/HOT WATER,999A KELLY STREET,1
136945,HEAT/HOT WATER,99A SOMERS STREET,9
136946,HEAT/HOT WATER,EAST 178 STREET,1
136947,HEAT/HOT WATER,EAST 207 STREET,1


In [14]:
df_Heat_Add.sort_values(by='frequency', ascending=False).head(5)

,Complaint_Type,Incident_Address,frequency
78534,HEAT/HOT WATER,34 ARDEN STREET,13858
130062,HEAT/HOT WATER,89-21 ELMHURST AVENUE,11548
85085,HEAT/HOT WATER,3810 BAILEY AVENUE,7115
2320,HEAT/HOT WATER,1025 BOYNTON AVENUE,5785
134607,HEAT/HOT WATER,9511 SHORE ROAD,5006


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #5: </h1>
<b>  <u>For the complaint types that you selected in the previous module</u> that had at least 800,000 compliants logged, <font color=blue>how many of the submitted tickets were closed?</font></b>
</div>

In [15]:
# Calculate the frequency by Complaint type and Borough
df_CT_Status = pd.DataFrame({'frequency': df_q2.groupby( by=['Complaint_Type', 'Status'] ).size()}).reset_index() # could use dataframe for a later use

# Calculate the frequency by Complaint type and Borough
df_Heat_Status = df_CT_Status[df_CT_Status['Complaint_Type'] == 'HEAT/HOT WATER']
df_Heat_Status.reset_index(drop=True, inplace=True)
df_Heat_Status

,Complaint_Type,Status,frequency
0,HEAT/HOT WATER,CLOSED,2133331
1,HEAT/HOT WATER,OPEN,16091
2,HEAT/HOT WATER,PENDING,2
